## **Laboratorio 9: El Método de Newton**
**Facultad de ingeniería, Departamento de Ingeniería Biomédica, Universidad de los Andes**\
**IBIO-2340: Fundamentos de Machine Learning**

**Nombres de los integrantes**


1.   Alejandro Narváez - 202123110
2.   Ali Cifci - 202210698

**Número del grupo**

LabML-4 10

# **Método de Newton**

El *Método de Newton*, al igual que el algoritmo de Descenso de Gradiente, es un método de búsqueda de línea utilizado para encontrar el mínimo de una función $f: \mathbb{R^n} \rightarrow \mathbb{R}$. A diferencia del Descenso de Gradiente, que emplea únicamente el gradiente como dirección de búsqueda, el Método de Newton utiliza la dirección dada por el producto de la inversa de la matriz Hessiana con el gradiente de la función. Esto le permite aprovechar información de la curvatura de $f$, logrando, en muchos casos, una convergencia más rápida.

**1. Fundamento teórico**

Tal como los demás métodos de búsqueda de línea, el método de Newton encuentra su fundamento teórico en el Teorema de Taylor:

*Teorema de Taylor:* Suponga que $f: \mathbb{R^n} \rightarrow \mathbb{R}$ es continuamente diferenciable y que $p \in \mathbb{R^n}$. Entonces tenemos que:

$$
f(x + p) = f(x) +  p^{T}  \nabla f(x + tp)
$$

Para algún $t \in (0,1)$. Más aún, si $f$ es dos veces continuamente diferenciable, tenemos que:

$$
f(x + p) = f(x) +  p^{T}  \nabla f(x + tp) + \frac{1}{2}p^{T} \nabla ^{2} f(x + tp)p
$$

Para algún $t \in (0,1)$. Así, *el teorema de Taylor nos permite construir una aproximación a la función alrededor de un punto $x$ en su dominio a partir del valor del gradiente y de la Hessiana de la función en dicho punto*. Dentro del contexto de la optimización numérica, $p$ se conoce como dirección de búsqueda y se elige de modo que:

$$
min_{(\alpha > 0)} f(x_{k} + \alpha p)
$$

Siendo $\alpha$ la longitud de paso o tasa de aprendizaje.

Al tomar como dirección de búsqueda $p$ aquella opuesta al gradiente de la función, recuperamos el algoritmo de Descenso de Gradiente que ya conocemos:

$$
x_{n+1} = x_{n} - \alpha  \nabla f(x) 
$$

Sin embargo, no es la única dirección de búsqueda que podemos definir. En primer lugar, vamos a suponer que podemos aproximar la función $f$ a través de una función modelo $m_{k}$, cuyo comportamiento cerca al punto actual $x_{k}$ es similar al de la función objetivo $f$ en dicho punto, $f_{k}$, y que esa función es cuadrática y de la forma:

$$
m_{k} (x_{k} + p) = f_{k} + p^{T} \nabla f_{k} + \frac{1}{2} p^{T} B_{k} p 
$$

Donde $f_k$, $\nabla f_k$ y $B_k$ es un escalar, un vector y una matriz respectivamente. *Nota que se parece demasiado a la expansión a segundo orden del Teorema de Taylor*, y no es gratuito, porque justamente vamos a asumir que ambas expresiones son iguales:

$$
f(x_{k} + p) \approx f_{k} +  p^{T}  \nabla f_{k} + \frac{1}{2}p^{T} \nabla ^{2} f_{k} p = m_{k}
$$

De esta forma, hallar la dirección de búsqueda que minimiza $f$ se reduce a hallar la dirección de búsqueda que minimiza a $m_{k}$, esto es, por supuesto, calcular el gradiente de $m_{k}$, igualar a cero y despejar $p$:

$$
\nabla m_{k} = \nabla [f_{k} + p^{T} \nabla f_{k} + \frac{1}{2} p^{T} B_{k} p] = 0
$$

$$
\nabla m_{k} = \nabla f_{k} + \nabla ^{2} f_{k} p^{T} = 0
$$

Es decir,

$$
p^{T} = - ( \nabla ^{2} f_{k} )^{-1} \nabla f_{k}
$$

Por lo tanto, al considerar la nueva dirección de búsqueda y asumiendo $\nabla ^{2} f_{k}$ es definida positiva (¿por qué esto es necesario?) tenemos que:

$$
x_{n+1} = x_{n} - ( \nabla ^{2} f_{k} )^{-1} \nabla f_{k}
$$

Esta expresión iterativa es lo que se conoce como *método de Newton*. Bajo esta alternativa, aproximamos a una forma cuadrática una región cercana al mínimo de la función. Conforme mejor se adapte la función objetivo a una forma cuadrática más preciso será el mínimo hallado. Note que en esta expresión no aparece la tasa de aprendizaje $\alpha$, esto se debe a que, por lo general, las implementaciones del método de Newton asumen $\alpha = 1$.


**2. Algoritmo del Método de Newton**

Veamos paso a paso como opera el algoritmo:

0. **Definimos un parámetro de convergencia $ϵ$ y un número máximo de iteraciones $N_{max}$**: El parámetro de convergencia nos ayuda a definir la precisión que se quiere alcanzar durante la búsqueda. El número máximo de iteraciones es un límite que evita que el algoritmo se quede estancado en un ciclo indeterminado y pueda ocasionar daños al procesador. Al cumplirse este límite el algoritmo debe detenerse inmediatamente.

1. **Iniciamos desde un punto de partida ($x_0$)**: Este punto inicial es arbitrario, un punto cualquiera en el dominio de la función. Su elección se hace crítica cuando la función no es convexa o tiene varios mínimos locales.

2. **Calculamos el gradiente y la matriz Hessiana para la función en este punto $\nabla_f(x_{0}), \nabla^{2} f(x_{0})$**: Calculamos el vector gradiente de la función objetivo $\nabla_{f}$ y su matriz Hessiana $\nabla^{2} f$ y los evaluamos en el punto de partida $\nabla_{f}(x_{0})$ y $\nabla^{2} f(x_{0})$.

3. **Calculamos la inversa de la matriz Hessiana $(\nabla^{2} f(x_{0}))^{-1}$**: Para esto debemos asegurarnos que la matriz Hessiana sea invertible lo cual se tiene si es definida positiva. 

4. **Hallamos un nuevo punto inicial ($x_1$)**: Al tomar la diferencia $x_0 - ( \nabla ^{2} f(x_0) )^{-1} \nabla f(x_0)$ damos un paso en dirección al mínimo que nos lleva a un nuevo punto $x_1$ que se encuentra más cerca de éste valor. Verificamos la condición de parada $||x_{1} - x_{0}||_2<ϵ$. Si se cumple, entonces tomamos $x_1$ como mínimo, si no continuamos al punto 5:

\begin{align*}
x_{1} = x_{0} - ( \nabla ^{2} f(x_{0}) )^{-1} \nabla f (x_{0})
\end{align*}


5. **Iteramos**: Tomando $x_{1}$ como nuevo punto inicial repetimos el proceso anterior, que continua siguiendo el algoritmo:

\begin{align*}
x_{n+1} = x_{n} -( \nabla ^{2} f(x_{n}) )^{-1} \nabla f(x_{n})
\end{align*}

Hasta que se cumpla la condición de parada $||x_{n+1} - x_{n}||_2<ϵ$. Cuando se cumple finaliza el método y tomamos el último valor $x_{n+1}$ como el mínimo de la función.

**3. Ventajas del Método de Newton**

- **Convergencia rápida**: Cuando la función objetivo es suave, convexa y dos veces diferenciable, el método de Newton converge mucho más rápido que el Descenso de Gradiente.

- **Tasa de Convergencia cuadrática**: Típicamente la tasa de convergencia del método de Newton es cuadrática, lo cual quiere decir que la precisión de la solución mejora exponencialmente en cada iteración.

- **No depende de un Hiperparámetro**: A diferencia del Descenso de Gradiente, el método de Newton aprovecha la información de curvatura de la función, contenida en la matriz Hessiana, para ajustar la dirección y tamaño del paso de búsqueda, lo cual evita problemas de escalado y elimina la necesidad de elegir una tasa de aprendizaje adecuada.


**3. Desventajas del Método de Newton**

- **Alto Costo Computacional**: Calcular la matriz Hessiana y su inversa en cada iteración tiene un costo computacional de $O(n^3)$, en comparación el método de Descenso de Gradiente tiene un costo computacional de $O(n)$. Por lo tanto, el Descenso de Gradiente es mucho más escalable en problemas con grandes cantidades de datos.

- **Depende de forma crítica de la matriz Hessiana**: Si la matriz Hessiana es singular no puede aplicarse directamente el método. Si tiene valores propios muy pequeños o muy grandes (esta mal condicionada) el método se torna inestable, puesto que en las direcciones en las que el gradiente coincida con los autovectores de la matriz Hessiana inversa los pasos serán muy pequeños o muy grandes haciendo que el algoritmo se estanque, oscile en torno al mínimo o diverja.

- **Funciones Complejas y No Convexas**: Si la función es no convexa, el método de Newton puede converger a un punto de silla en lugar de un mínimo. El método de Newton es ideal para funciones cuadráticas o que localmente sigan un comportamiento cuadrático, en funciones con comportamientos distintos puede requerir de más iteraciones para converger debido a la pérdida de precisión de la aproximación.

**1.1 Ejercicios**

El objetivo de esta práctica es comparar el método de Newton con el algoritmo de Descenso de Gradiente minimizando una función de prueba con una condición de parada previamente definida. Para esto, tome la siguiente expresión como función objetivo:

$$
 f(x) = (1-x_1)^2 + 5(x_2-x_1^2)^2,
$$

donde $x \in \mathbb{R^2}$. 



1. Encuentre  analíticamente el vector gradiente y la matriz Hessiana para la función objetivo y escribalos a continuación:

2. Implemente el método de Newton teniendo en cuenta la condición de parada y haciendo uso de los siguientes parámetros:

 - $\epsilon =0.01$
 - $x[0]=[0,0]^T$
 - $N_{max} = 100$


3. En tres subplots diferentes de una misma figura grafique:

*  La trayectoria de búsqueda del mínimo sobre las curvas de nivel de la función objetivo.
*  El valor la función objetivo en cada iteración vs el número de iteración $k$.
*  El valor de $||x[k+1]-x[k]||_2$ vs el número de iteración $k$.



In [ ]:
# Celda para agregar el código de las gráficas

4. Use el algoritmo de Descenso de Gradiente implementado en el laboratorio anterior para encontrar el mínimo de la función objetivo, tome $\alpha =0.05$ y los valores anteriores de $\epsilon$, $N_{max}$ y $x[0]$.

5. De acuerdo a los resultados obtenidos con el algoritmo de Descenso de Gradiente del punto anterior, en tres subplots diferentes de una misma figura grafique:

*  La trayectoria de búsqueda del mínimo sobre las curvas de nivel de la función objetivo.
*  El valor la función objetivo en cada iteración vs el número de iteración $k$.
*  El valor de $||x[k+1]-x[k]||_2$ vs el número de iteración $k$.

6. Responda las siguientes preguntas:

- ¿Por qué cree que al algoritmo de Descenso de Gradiente le cuesta más converger que al método de Newton?

- Note el grado del polinómio de la función, ¿para qué grado polinómico el método de Newton converge en una sola iteración?

- ¿Cree que el punto inicial influye en la convergencia de ambos algoritmos?

- ¿Cómo podría mejorar la eficiencia del algoritmo de Descenso de Gradiente?